## Lab Work 2 : Linear Regression

This notebook builds on the second lecture of Foundations of Machine Learning. We'll focus on the linear regression model.

Important note: the steps shown here are not always the most efficient or the most "industry-approved." Their main purpose is pedagogical. So don't panic if something looks suboptimal—it's meant to be.

If you have questions (theoretical or practical), don't hesitate to bug your lecturer.

## Introduction

In this sequence, we are trying to predict the price of the stock TSM : a taiwanese manufacturer of GPU chips, mostly used nowadays in AI. Let's have a look at the features we have to do it.

In [ ]:
import pandas as pd

df = pd.read_csv("TSM.csv")
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)
df.head()


All of the above data comes from the [Yahoo Finance Python API](https://github.com/ranaroussi/yfinance). It gives access to all finance market data live listed on [Yahoo Finance](https://fr.finance.yahoo.com/). We choosed the daily close price of :
* **Companies** : Taiwan Semiconductor Manufacturing ([TSM](https://finance.yahoo.com/quote/TSM/)), Nvidia ([NVDA](https://finance.yahoo.com/quote/NVDA/)) and Micron Technology ([MU](https://finance.yahoo.com/quote/MU/))
* **Comodities** : Gold, Silver, Platinium, Paladium and Coppper. We couldn't get the silicon price because it is not freely available.

Our hypothese is that theses tickers can help use predict TSM's stock price. Let's have a look first at his price through time :



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style(style="whitegrid")

plt.figure(figsize=(12, 6))
plt.axvline(x=pd.to_datetime("2022-11-30"), ls='--', color="black", label="ChatGPT released", lw=1)
plt.plot(df["TSM"], label="TSM")
plt.legend()
plt.show()

## Data exploration

We need to go a bit deeper.

**Task** : Use the [`scatter_matrix`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html) function to start exploration relationship between all the columns.

**Task** : Use the [`heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) function with the [`corr`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) method on the dataframe and plot the result.

**Task** : Given the previous cells, make a choice on which columns to keep.

## Feature engineering

Before we continue, we need to split the training and test set. In our setup, doing it randomly with [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) will lead to data leakage. Indeed, we might train on future data and test on data from the past, so we need to split according to time.

**Task** : Define a `train_test_time_splitting` function that split a matrix of feature *X* and a target vector *y* according to a *train_ratio*. 


Now we need to build time series specific features. As we have daily closing price, we can't used them to predict *directly*. Instead, we can compute features based on the past values :

* **Lags** : the last value for a given period. One can use the [`shift`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html) method for a vector.
* **Rolling** : the average or standard deviation of the previous values for a given period. One can use the [`rolling`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html) method for a vector alongside the `mean` and `std` functions.
* **Return** : the evolution across a given period. One can use the [`pct_change`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) method for a vector.

**Task** : Define a function that build lags, rolling and return features (each being optionnal) for a dataframe and specifics columns. Also specify the periods length.



**Task** : Use the previous function to compute rolling features of your choice.

## Modelisation

Let's recap what we have learned so far, including the previous session :
1. A linear regression needs its input to be float and standardized : we use the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to this end
2. It is easier to wrap the previous step with learning using a [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
3. We need to split the dataset into train and test set according to time, using our own `train_test_time_splitting` function
4. We shall have a way to compare our work to a strong baseline. Here we will take the rolling average of TSM stock value in the last 7 days.


Common metrics to assess the performance of a regressor are, given a dataset of $n$ sample, ground truth vector $y$ and predicted values vector $\hat{y}$:
* **Root Mean Square Error** ([`RMSE`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html)) : penalize more large errors than MAE
$$ RMSE(y, \hat{y}) = \sqrt{\frac{1}{n} \sum_{k=1}^n (y_k - \hat{y}_k)^2}$$

* **Mean Absolute Error** ([`MAE`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html)) : penalize smaller errors more than RMSE, all errors *weights* the same
$$ MAE(y, \hat{y}) = \frac{1}{n}\sum_{k=1}^n \left|y_k - \hat{y}_k\right|$$

* **[$R^2$](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) Score** : bounded above by 1, gives insights on how well the model grasps the *structure*. Note that $R^2$ can be negative.
$$ R^2(y, \hat{y}) = 1 - \frac{\displaystyle \sum_{k=1}^n (y_k - \hat{y_k})^2}{\displaystyle \sum_{k=1}^n (y_k - \overline{y})^2}$$


**Task** : Define a function that takes a dataset with features, the target column and the features columns. Then the function will split, train and display performance on the test according to the three above metrics, and display the performance of the baseline.

**Task** : use this function for our purpose using only TSM and NVDA rolling avg features.

Our model performs worse than the rolling average baseline, yet showing great metric performance. We need to improve our model.

## Doing better

**Task** : Run the following cell and analyze its output.

We see that the trend between TSM stock closing price and NVDA's are not linearly related, but they are with a root square ! Therefore, to *help* the linear model, we shall switch from *pure* NVDA's stock values to square root values. Obviously, one need to check that it works with all rolling average column related to NVIDIA.

**Task** : Implement the described idea. Then, train again a model using these features.

Now, this is better !

**Task** : Keep trying ideas to outperform even more the baseline.